## NLP
## Lab 5 - Morphosyntactic tagging

In [60]:
import requests
import os
from collections import Counter
import math

In [12]:
print(requests.post('http://localhost:9200', data="Ala ma kota.").text)

Ala	none
	Ala	subst:sg:nom:f	disamb
ma	space
	mieć	fin:sg:ter:imperf	disamb
kota	space
	kot	subst:sg:acc:m2	disamb
.	none
	.	interp	disamb




In [13]:
all_files = []
path = './ustawy'
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r', encoding='utf-8') as file:
        all_files.append(" ".join(file.read().lower().replace('\n', ' ').split()))

#### Zadanie 2.

In [20]:
tag_lem_files = [requests.post('http://localhost:9200', data=file.encode('utf-8')).text for file in all_files]

In [31]:
lines = tag_lem_files[0].split('\n')
for line in lines[50:70]:
    print(line.split('\t'))

['o', 'space']
['', 'o', 'prep:loc', 'disamb']
['podatku', 'space']
['', 'podatek', 'subst:sg:loc:m3', 'disamb']
['od', 'space']
['', 'od', 'prep:gen:nwok', 'disamb']
['towarów', 'space']
['', 'towar', 'subst:pl:gen:m3', 'disamb']
['i', 'space']
['', 'i', 'conj', 'disamb']
['usług', 'space']
['', 'usługa', 'subst:pl:gen:f', 'disamb']
['oraz', 'space']
['', 'oraz', 'conj', 'disamb']
['o', 'space']
['', 'o', 'prep:loc', 'disamb']
['podatku', 'space']
['', 'podatek', 'subst:sg:loc:m3', 'disamb']
['akcyzowym', 'space']
['', 'akcyzowy', 'adj:sg:loc:m3:pos', 'disamb']


Pod uwagę należy wziąć wyłącznie czteroelementowe linie. Potrzebne informacje to: zlematyzowane słowo znajdujące się pod indeksem 1 oraz kategoria morfosyntaktyczna znajdująca się przed pierwszym dwukropkiem w elemencie o indeksie 2.

In [35]:
lemmatized_files = []
for file in tag_lem_files:
    tokens = []
    lines = file.split('\n')
    for line in lines:
        split_line = line.split('\t')
        if len(split_line) == 4:
            word = split_line[1]
            category = split_line[2].split(':')[0]
            tokens.append((word, category))
    lemmatized_files.append(tokens)

In [36]:
lemmatized_files[0][20:30]

[('rok', 'brev'),
 ('.', 'interp'),
 ('o', 'prep'),
 ('zmiana', 'subst'),
 ('ustawa', 'subst'),
 ('o', 'prep'),
 ('podatek', 'subst'),
 ('od', 'prep'),
 ('towar', 'subst'),
 ('i', 'conj')]

#### Zadanie 3.

Bigramy przetwarzane będą w postaci par składających się z par ((słowo, kategoria), (słowo, kategoria)).

In [64]:
bigrams_counts = {}
for file in lemmatized_files:
    bigrams = [(file[start_pos], file[start_pos+1]) for start_pos in range(len(file) - 2 + 1)]
    bigrams_counter = Counter(bigrams)
    for bigram, count in bigrams_counter.items():
        if bigram in bigrams_counts:
            bigrams_counts[bigram] += count
        else:
            bigrams_counts[bigram] = count

In [45]:
def get_entries_range(dictionary, start, end):
    return {k: dictionary[k] for k in list(dictionary)[start:end]}

In [49]:
get_entries_range(bigrams_counts,0,15)

{(('dziennik', 'brev'), ('.', 'interp')): 8881,
 (('.', 'interp'), ('u', 'prep')): 5708,
 (('u', 'prep'), ('.', 'interp')): 5668,
 (('.', 'interp'), ('z', 'prep')): 5478,
 (('z', 'prep'), ('1993', 'adj')): 536,
 (('1993', 'adj'), ('rok', 'brev')): 787,
 (('rok', 'brev'), ('.', 'interp')): 33025,
 (('.', 'interp'), ('numer', 'brev')): 21424,
 (('numer', 'brev'), ('129', 'num')): 137,
 (('129', 'num'), (',', 'interp')): 137,
 ((',', 'interp'), ('pozycja', 'brev')): 42998,
 (('pozycja', 'brev'), ('.', 'interp')): 45044,
 (('.', 'interp'), ('599', 'adj')): 8,
 (('599', 'adj'), ('ustawa', 'subst')): 1,
 (('ustawa', 'subst'), ('z', 'prep')): 8625}

#### Zadanie 4.

In [43]:
def is_bigram_alpha(bigram):
    return bigram[0][0].isalpha() and bigram[1][0].isalpha()

In [47]:
letters_bigrams_counts = {bigram : count for bigram, count in bigrams_counts.items() if is_bigram_alpha(bigram)}

In [48]:
get_entries_range(letters_bigrams_counts,0,10)

{(('ustawa', 'subst'), ('z', 'prep')): 8625,
 (('z', 'prep'), ('dzień', 'subst')): 11360,
 (('o', 'prep'), ('zmiana', 'subst')): 1409,
 (('zmiana', 'subst'), ('ustawa', 'subst')): 908,
 (('ustawa', 'subst'), ('o', 'prep')): 1668,
 (('o', 'prep'), ('podatek', 'subst')): 592,
 (('podatek', 'subst'), ('od', 'prep')): 479,
 (('od', 'prep'), ('towar', 'subst')): 274,
 (('towar', 'subst'), ('i', 'conj')): 615,
 (('i', 'conj'), ('usługa', 'subst')): 642}

In [50]:
sorted_letters_bigrams_counts = {k: v for k, v in sorted(letters_bigrams_counts.items(), key=lambda item: item[1]*-1)}

In [51]:
get_entries_range(sorted_letters_bigrams_counts,0,10)

{(('w', 'prep'), ('artykuł', 'brev')): 31972,
 (('o', 'prep'), ('który', 'adj')): 28656,
 (('który', 'adj'), ('mowa', 'subst')): 28538,
 (('mowa', 'subst'), ('w', 'prep')): 28473,
 (('w', 'prep'), ('ustęp', 'brev')): 23501,
 (('z', 'prep'), ('dzień', 'subst')): 11360,
 (('otrzymywać', 'fin'), ('brzmienie', 'subst')): 10532,
 (('określić', 'ppas'), ('w', 'prep')): 9772,
 (('do', 'prep'), ('sprawa', 'subst')): 8718,
 (('ustawa', 'subst'), ('z', 'prep')): 8625}

#### Zadanie 6.

Do obliczenia LLR przydatne będą również ilosci wystąpień pojedynczych zlematyzowanych słów.

In [54]:
lemmas_counts = {}
for file in lemmatized_files:
    lemmas_counter = Counter([lemma for lemma in file])
    for lemma, count in lemmas_counter.items():
        if lemma in lemmas_counts:
            lemmas_counts[lemma] += count
        else:
            lemmas_counts[lemma] = count

In [55]:
get_entries_range(lemmas_counts,0,10)

{('dziennik', 'brev'): 8881,
 ('.', 'interp'): 457514,
 ('u', 'prep'): 6687,
 ('z', 'prep'): 87989,
 ('1993', 'adj'): 800,
 ('rok', 'brev'): 33127,
 ('numer', 'brev'): 44940,
 ('129', 'num'): 157,
 (',', 'interp'): 343058,
 ('pozycja', 'brev'): 45044}

In [58]:
def shannon_entropy(k_list):
    n = sum(k_list)
    return sum([k / n * math.log(abs(k) / n + (k == 0)) for k in k_list])

all_bigrams_occ = sum(letters_bigrams_counts.values())
def llr(bigram):
    k11 = letters_bigrams_counts[bigram]
    k21 = lemmas_counts[bigram[0]] - letters_bigrams_counts[bigram]
    k12 = lemmas_counts[bigram[1]] - letters_bigrams_counts[bigram]
    k22 = all_bigrams_occ - lemmas_counts[bigram[0]] - lemmas_counts[bigram[1]]
    return 2 * sum([k11, k12, k21, k22]) * (shannon_entropy([k11, k12, k21, k22]) - shannon_entropy([k11 + k12, k21 + k22]) - shannon_entropy([k11 + k21, k12 + k22]))

In [61]:
bigrams_llr = {bigram: llr(bigram) for bigram in letters_bigrams_counts.keys()}

In [62]:
get_entries_range(bigrams_llr, 0, 10)

{(('ustawa', 'subst'), ('z', 'prep')): 29011.42501465483,
 (('z', 'prep'), ('dzień', 'subst')): 42895.817726105815,
 (('o', 'prep'), ('zmiana', 'subst')): 3769.4023989730326,
 (('zmiana', 'subst'), ('ustawa', 'subst')): 3243.676314830327,
 (('ustawa', 'subst'), ('o', 'prep')): 1392.9696014781416,
 (('o', 'prep'), ('podatek', 'subst')): 1571.6179206343165,
 (('podatek', 'subst'), ('od', 'prep')): 2294.323585353027,
 (('od', 'prep'), ('towar', 'subst')): 863.6277570545774,
 (('towar', 'subst'), ('i', 'conj')): 1031.2056580868837,
 (('i', 'conj'), ('usługa', 'subst')): 1504.0659194013892}

#### Zadanie 7.

In [70]:
partitions = {}
for bigram, count in letters_bigrams_counts.items():
    key = (bigram[0][1], bigram[1][1])
    if not key in partitions:
        partitions[key] = {}
    partitions[key][bigram] = count

In [71]:
partitions_counts = {partition : sum(bigram_counts.values()) for partition, bigram_counts in partitions.items()}

In [75]:
sorted_partitions_counts = {k: v for k, v in sorted(partitions_counts.items(), key=lambda item: item[1]*-1)}

#### Zadanie 8.

In [76]:
get_entries_range(sorted_partitions_counts, 0, 10)

{('prep', 'subst'): 323591,
 ('subst', 'subst'): 280316,
 ('subst', 'adj'): 273260,
 ('adj', 'subst'): 188028,
 ('subst', 'prep'): 171005,
 ('subst', 'conj'): 84094,
 ('conj', 'subst'): 83052,
 ('ger', 'subst'): 81237,
 ('prep', 'adj'): 79621,
 ('prep', 'brev'): 66983}

#### Zadanie 9.

In [77]:
sorted_bigrams_llr = {k: v for k, v in sorted(bigrams_llr.items(), key=lambda item: item[1]*-1)}

In [81]:
largest_partitions = get_entries_range(sorted_partitions_counts, 0, 10).keys()
for partition in largest_partitions:
    print('\nPARTITION: {} {}'.format(partition[0], partition[1]))
    found_bigrams = 0
    for bigram, llr in sorted_bigrams_llr.items():
        if partition[0] == bigram[0][1] and partition[1] == bigram[1][1]:
            found_bigrams += 1
            print('\t{} {} (LLR: {})'.format(bigram[0][0], bigram[1][0], llr))
        if found_bigrams == 5:
            break


PARTITION: prep subst
	do sprawa (LLR: 45158.814783427115)
	na podstawa (LLR: 45117.47349958786)
	z dzień (LLR: 42895.817726105815)
	w droga (LLR: 31254.20111151821)
	od dzień (LLR: 29261.80012336015)

PARTITION: subst subst
	droga rozporządzenie (LLR: 51600.13583315287)
	skarb państwo (LLR: 21954.86897255017)
	rada minister (LLR: 14640.701336068578)
	terytorium rzeczpospolita (LLR: 14062.158500531574)
	ochrona środowisko (LLR: 13824.170027843027)

PARTITION: subst adj
	minister właściwy (LLR: 67532.06692804613)
	rzeczpospolita polski (LLR: 43082.50424659325)
	jednostka organizacyjny (LLR: 23877.483005369744)
	samorząd terytorialny (LLR: 23183.83713912349)
	produkt leczniczy (LLR: 21276.14344846115)

PARTITION: adj subst
	który mowa (LLR: 246775.97859879132)
	niniejszy ustawa (LLR: 19824.505355383506)
	następujący zmiana (LLR: 19331.98773640784)
	odrębny przepis (LLR: 11680.78807459265)
	walny zgromadzenie (LLR: 9437.430074982918)

PARTITION: subst prep
	mowa w (LLR: 148360.1239479386

#### Zadanie 10.

Odnalezione bigramy z najwyższymi wynikami LLR w każdej kategorii są złożeniami typowymi dla tekstów prawnych.  
*Multiword expressions* mające same w sobie jakieś znaczenie znalazły się głównie w kategoriach *subst subst* oraz *subst adj*, a więc są to bigramy złożone z dwóch rzeczowników lub z rzeczownika, po którym następuje przymiotnik. W kategorii *adj subst* (przymiotnik przed rzeczownikiem) znalazły się także wartościowe *multiword expressions*, jednak nawet w pierwszej piątce nie wszystkie same coś znaczą (np. który mowa).  
Do rozpoznawania *multiword expressions* lepszym rozwiązaniem wydaje się być zastosowanie podziału na kategorie syntaktyczne, niż używanie samego LLR, ponieważ znaczące *multiword expressions* najczęściej nie otrzymają najwyższego wyniku LLR. LLR może być wskaźnikiem pomocniczym do obliczenia najbardziej reprezentatywnych wyrażeń dla poszczególnych kategorii.  
Kategorie morfosyntaktyczne mogą być wykorzystywane np. do wyznaczania różnych kontekstów, w których słowo może być używane oraz rozpoznawania różnych jego znaczeń.